In [1]:
import numpy as np
from sklearn import preprocessing
import scipy.stats

from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.cluster import KMeans

In [2]:
#preencher nome da rede geodesica analisada (abaixo os nomes das redes A, B, C, D, E e F, respectivamente)
#alias opcoes = {'gemael147','ghi355gps','nive20obs','gnssrbc21','ghiniv255','kleincorr'}
alias='gemael147'
aux_filename_36_train="_0-2outs_3-6s_50000itr_train"
aux_filename_69_train="_0-2outs_6-9s_50000itr_train"
aux_filename_36="_0out200000_1-2outs_3-6s_50000itr"
aux_filename_69="_0out200000_1-2outs_6-9s_50000itr"
aux_filename_915="_0-2outs_9-15s_10000itr"
aux_filename_36_3out="_0-3outs_3-6s_10000itr"
aux_filename_69_3out="_0-3outs_6-9s_10000itr"

In [3]:
#importa arquivos para treinamento do k-médias
caminho="C:/Users/Patricia/Desktop/IP3new/00.treino_rna/"+alias+"_" 
A = np.loadtxt(caminho+"A")
m = A.shape[0]
qtt_exs = m*150000

wAllEL1_pi_sigma_v_36_train = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_36_train).reshape(qtt_exs,1)
wAllELInf_pp_sig_obs_36_train = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_36_train).reshape(qtt_exs,1)
wAllELInf_pi_sig_obs_36_train = np.loadtxt(caminho+"wAllELInf_pi_sig_obs"+aux_filename_36_train).reshape(qtt_exs,1)
wAllIDS_pp_36_train = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_36_train).reshape(qtt_exs,1)
wAllSLRTMO_pp_36_train = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_36_train).reshape(qtt_exs,1)
outs_positions_36_train = np.loadtxt(caminho+"outs_positions"+aux_filename_36_train).reshape(qtt_exs,1)

wAllEL1_pi_sigma_v_69_train = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_69_train).reshape(qtt_exs,1)
wAllELInf_pp_sig_obs_69_train = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_69_train).reshape(qtt_exs,1)
wAllELInf_pi_sig_obs_69_train = np.loadtxt(caminho+"wAllELInf_pi_sig_obs"+aux_filename_69_train).reshape(qtt_exs,1)
wAllIDS_pp_69_train = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_69_train).reshape(qtt_exs,1)
wAllSLRTMO_pp_69_train = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_69_train).reshape(qtt_exs,1)
outs_positions_69_train = np.loadtxt(caminho+"outs_positions"+aux_filename_69_train).reshape(qtt_exs,1)

In [4]:
#importa arquivos para testes do k-médias
caminho="C:/Users/Patricia/Desktop/IP3new/"+alias+"_" 
qtt_exs = m*300000

wAllEL1_pi_sigma_v_36 = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_36).reshape(qtt_exs,1)
wAllELInf_pp_sig_obs_36 = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_36).reshape(qtt_exs,1)
wAllELInf_pi_sig_obs_36 = np.loadtxt(caminho+"wAllELInf_pi_sig_obs"+aux_filename_36).reshape(qtt_exs,1)
wAllIDS_pp_36 = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_36).reshape(qtt_exs,1)
wAllSLRTMO_pp_36 = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_36).reshape(qtt_exs,1)
outs_positions_36 = np.loadtxt(caminho+"outs_positions"+aux_filename_36).reshape(qtt_exs,1)

wAllEL1_pi_sigma_v_69 = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_69).reshape(qtt_exs,1)
wAllELInf_pp_sig_obs_69 = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_69).reshape(qtt_exs,1)
wAllELInf_pi_sig_obs_69 = np.loadtxt(caminho+"wAllELInf_pi_sig_obs"+aux_filename_69).reshape(qtt_exs,1)
wAllIDS_pp_69 = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_69).reshape(qtt_exs,1)
wAllSLRTMO_pp_69 = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_69).reshape(qtt_exs,1)
outs_positions_69 = np.loadtxt(caminho+"outs_positions"+aux_filename_69).reshape(qtt_exs,1)

In [5]:
#forma conjuntos de treino, validacao e testes
qtd_graphs = 30000 
#qtd de MC cenários de redes geodésicas para cada qtd de outliers e intervalo de magnitude para o treinamento
#a qtd de exemplos de treinamento vai ser isso x5, pois entra 0, 1 e 2 outs (sendo 1 e 2 outs para 3-6 e 6-9sigma); 

#######train
aux36_train = (wAllIDS_pp_36_train,
             wAllEL1_pi_sigma_v_36_train,
             wAllELInf_pi_sig_obs_36_train,wAllELInf_pp_sig_obs_36_train,
             wAllSLRTMO_pp_36_train)########################## tirar redun sfc
obs_36_train = np.concatenate(aux36_train,axis=1)
label_36_train = outs_positions_36_train

aux69_train = (wAllIDS_pp_69_train,
             wAllEL1_pi_sigma_v_69_train,
             wAllELInf_pi_sig_obs_69_train,wAllELInf_pp_sig_obs_69_train,
             wAllSLRTMO_pp_69_train)########################## tirar redun sfc
obs_69_train = np.concatenate(aux69_train,axis=1) 
label_69_train = outs_positions_69_train

#####teste
aux36 = (wAllIDS_pp_36,
             wAllEL1_pi_sigma_v_36,
             wAllELInf_pi_sig_obs_36,wAllELInf_pp_sig_obs_36,
             wAllSLRTMO_pp_36)########################## tirar redun sfc
X_test36 = np.concatenate(aux36,axis=1)
y_test36 = outs_positions_36.flatten('C')

aux69 = (wAllIDS_pp_69,
             wAllEL1_pi_sigma_v_69,
             wAllELInf_pi_sig_obs_69,wAllELInf_pp_sig_obs_69,
             wAllSLRTMO_pp_69)########################## tirar redun sfc
X_test69 = np.concatenate(aux69,axis=1) 
y_test69 = outs_positions_69.flatten('C')

              
prop2out = 1
X_train36 = np.concatenate((obs_36_train[0:0+m*qtd_graphs,:],obs_36_train[m*50000:m*50000+m*qtd_graphs,:],
                         obs_36_train[m*100000:m*100000+int(m*qtd_graphs*prop2out),:]),axis=0)
y_train36 = np.concatenate((label_36_train[0:0+m*qtd_graphs,:],label_36_train[m*50000:m*50000+m*qtd_graphs,:],
                         label_36_train[m*100000:m*100000+int(m*qtd_graphs*prop2out),:]),axis=0)
X_train69 = np.concatenate((obs_69_train[m*50000:m*50000+m*qtd_graphs,:],
                         obs_69_train[m*100000:m*100000+int(m*qtd_graphs*prop2out),:]),axis=0)
y_train69 = np.concatenate((label_69_train[m*50000:m*50000+m*qtd_graphs,:],
                         label_69_train[m*100000:m*100000+int(m*qtd_graphs*prop2out),:]),axis=0)
X_val36 = np.concatenate((obs_36_train[m*30000:m*50000,:],obs_36_train[m*80000:m*100000,:],obs_36_train[m*130000:m*150000,:]),axis=0)
X_val69 = np.concatenate((obs_69_train[m*30000:m*50000,:],obs_69_train[m*80000:m*100000,:],obs_69_train[m*130000:m*150000,:]),axis=0)
y_val36 = np.concatenate((label_36_train[m*30000:m*50000,:],label_36_train[m*80000:m*100000,:],label_36_train[m*130000:m*150000,:]),axis=0).flatten('C')
y_val69 = np.concatenate((label_69_train[m*30000:m*50000,:],label_69_train[m*80000:m*100000,:],label_69_train[m*130000:m*150000,:]),axis=0).flatten('C')

X_train = np.concatenate((X_train36, X_train69),axis=0)
y_train = np.concatenate((y_train36, y_train69),axis=0).flatten('C')
X_val = np.concatenate((X_val36, X_val69),axis=0)
y_val = np.concatenate((y_val36, y_val69),axis=0).flatten('C')


print(X_train.shape,y_val.shape)

(2400000, 5) (1920000,)


In [6]:
def clf_eval(y_pred, y_val):
  if y_pred.ndim < 2 : ## isso eh apenas para o caso de granularidade nas obs..
    aux = int(y_pred.shape[0]/m)
    y_pred = np.reshape(y_pred, (aux, m), order='C')
    y_val = np.reshape(y_val, (aux, m), order='C')
  acc = 100*accuracy_score(y_pred, y_val)
  print('Accuracy: {}'.format(acc))
  #acurácia somente para conjunto que tenha 3 outliers 
  aux=sum(np.transpose(y_val))==3
  if np.any(aux):
      acc3=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
      print('Accuracy 3 out: {}'.format(acc3))
      acc2 = float("nan")
      acc1 = float("nan")
  #acurácia somente para conjunto que tenha 2 outliers 
  aux=sum(np.transpose(y_val))==2
  if np.any(aux):
      acc2=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
      print('Accuracy 2 out: {}'.format(acc2))
  #acurácia somente para conjunto que tenha 1 outliers 
  aux=sum(np.transpose(y_val))==1
  if np.any(aux):
      acc1=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
      print('Accuracy 1 out: {}'.format(acc1))
  #acurácia somente para conjunto que tenha 0 outliers 
  aux=sum(np.transpose(y_val))==0
  acc0=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
  print('Accuracy 0 out: {}'.format(acc0))
  acc = (acc2+acc1+acc0)/3
  print('MSR: {}'.format(acc))
  acc = [acc, acc1, acc2]
  return acc

In [7]:
def unsupervised_eval(config_kmeans,y_pred, y, n):
    c1 = Counter(y_pred)
    c1np = np.array(list(c1.items()))
    c1p = [(i, c1[i] / len(y_pred) * 100.0) for i, count in c1.most_common()]
    print(c1p)
    
#fazer que maior cluster seja predicao "0", sfc
    if c1np[0,0] != 0:
        aux=y_pred==c1np[0,0]
        y_pred[aux] = 1000
        aux=y_pred==0
        y_pred[aux] = c1np[0,0]
        aux=y_pred==1000
        y_pred[aux] = 0

    c1 = Counter(y_pred)
    c1p = [(i, c1[i] / len(y_pred) * 100.0) for i, count in c1.most_common()]
    print(c1p)
    #c2 = Counter(y)
    #c2p = [(i, c2[i] / len(y) * 100.0) for i, count in c2.most_common()]
    #print(c2p)

#juntar clusters   
    if config_kmeans != "2":  #se fosse 2 - maior cluster: não outlier/menor cluster: outlier.
        print("ok")
        if config_kmeans == "3a": #maior cluster: não outlier/ 2º maior cluster e menor cluster: outlier
           aux=y_pred==2
           y_pred[aux] = 1 
        if config_kmeans == "3b": #maior e 2º maior clusters: não outlier/ menor cluster: outlier
           aux=y_pred==1
           y_pred[aux] = 0
           aux=y_pred==2
           y_pred[aux] = 1 
        if config_kmeans == "3c": #maior e menor clusters: não outlier/ 2º maior cluster: outlier
           aux=y_pred==2
           y_pred[aux] = 0 

    c1 = Counter(y_pred)
    c1p = [(i, c1[i] / len(y_pred) * 100.0) for i, count in c1.most_common()]
    print(c1p)

    acc = 100*accuracy_score(y_pred, y)
    print (acc)
    clf_eval(y_pred, y)

In [8]:
#K_MEANS COM 2 CLUSTERS
n_clusters=2 
kmeans=KMeans(n_clusters,random_state=42)
kmeans.fit(X_train)

KMeans(n_clusters=2, random_state=42)

In [9]:
#kmedias-2
config_kmeans = "2"
print("CJ VALIDACAO:") 
print("3-9sigma:") 
y_pred_kmeans_val = kmeans.predict(X_val)
unsupervised_eval(config_kmeans,y_pred_kmeans_val, y_val, n_clusters)

CJ VALIDACAO:
3-9sigma:
[(1, 93.85145833333334), (0, 6.148541666666667)]
[(0, 93.85145833333334), (1, 6.148541666666667)]
[(0, 93.85145833333334), (1, 6.148541666666667)]
93.28729166666668
Accuracy: 37.622499999999995
Accuracy 2 out: 1.7975
Accuracy 1 out: 11.275
Accuracy 0 out: 99.795
MSR: 37.6225


In [10]:
# daqui em diante eh para salvar as predições com k=2 (melhor desempenho) para usar na metaclassificação 
save = 1

In [11]:
print("----------CJ TESTE:") 
print("3-6sigma:") 
y_pred_kmeans36 = kmeans.predict(X_test36)
unsupervised_eval(config_kmeans,y_pred_kmeans36, y_test36, n_clusters)
print("6-9sigma:") 
y_pred_kmeans69 = kmeans.predict(X_test69)
unsupervised_eval(config_kmeans,y_pred_kmeans69, y_test69, n_clusters)

if save==1: 
  caminho="C:/Users/Patricia/Desktop/IP3new/"+alias+"_"
  aux = int(y_pred_kmeans36.shape[0]/m)
  y_pred_kmeans36 = np.reshape(y_pred_kmeans36, (aux, m), order='C')
  y_pred_kmeans36 = np.concatenate((np.arange(m).reshape(1,m),y_pred_kmeans36)).astype(int)
  y_pred_kmeans69 = np.reshape(y_pred_kmeans69, (aux, m), order='C')
  y_pred_kmeans69 = np.concatenate((np.arange(m).reshape(1,m),y_pred_kmeans69)).astype(int)
  np.savetxt(caminho+'ypredkmeans'+'_3-6s'+'.csv', y_pred_kmeans36,delimiter=',',fmt='%i')
  np.savetxt(caminho+'ypredkmeans'+'_6-9s'+'.csv', y_pred_kmeans69,delimiter=',',fmt='%i')

----------CJ TESTE:
3-6sigma:
[(1, 98.6498125), (0, 1.3501875)]
[(0, 98.6498125), (1, 1.3501875)]
[(0, 98.6498125), (1, 1.3501875)]
96.9788125
Accuracy: 68.25233333333334
Accuracy 2 out: 1.902
Accuracy 1 out: 8.352
Accuracy 0 out: 99.815
MSR: 36.68966666666667
6-9sigma:
[(1, 95.20572916666666), (0, 4.794270833333333)]
[(0, 95.20572916666666), (1, 4.794270833333333)]
[(0, 95.20572916666666), (1, 4.794270833333333)]
96.29989583333334
Accuracy: 69.27833333333334
Accuracy 2 out: 1.856
Accuracy 1 out: 14.554
Accuracy 0 out: 99.815
MSR: 38.74166666666667
